In [13]:
import os

In [14]:
%pwd

'f:\\Anaconda Projects\\end to end wine quality fifth\\end-to-end-wine-quality-fifth'

In [16]:
from pathlib import Path
os.chdir(Path('F:\Anaconda Projects\end to end wine quality sixth\end-to-end-wine-quality-fifth'))

In [17]:
%pwd

'F:\\Anaconda Projects\\end to end wine quality sixth\\end-to-end-wine-quality-fifth'

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    # alpha: float
    # l1_ration: float
    target_column: str
    model_to_loop: str
    model_params: str
    test_array_path: Path
    train_array_path: Path

In [19]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

[2024-01-26 11:33:43,463: INFO: utils: NumExpr defaulting to 4 threads.]


In [20]:
import joblib
from ensure import ensure_annotations
from pathlib import Path
def save_object(file_path, obj):
    try:
        dir_path = os.path.dirname(file_path)
        os.makedirs(dir_path, exist_ok=True)

        with open(file_path, 'wb') as file_obj:
            joblib.dump(obj, file_obj)

    except Exception as e:
        raise e
@ensure_annotations
def load_object(file_path: Path):
    try:
        with open(file_path, 'rb') as file_obj:
            return joblib.load(file_obj)
    except Exception as e:
        raise e


In [21]:
from mlProject import *
from box import ConfigBox
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV

def evaluate_models(x_train, x_test, y_train, y_test, models: ConfigBox, params) -> dict:
    try:
        report = {}
        logger.info(f"Entered to model evaluation list")

        for i in range(len(list(models))):
            model = eval(list(models.values())[i])
            
            param = params[list(models.keys())[i]]

            rs = RandomizedSearchCV(model, param)

            rs.fit(x_train, y_train)

            # model.fit(x_train, y_train)

            model.set_params(**rs.best_params_)
            model.fit(x_train, y_train)

            y_test_pred = model.predict(x_test)

            test_model_r2_score = r2_score(y_test, y_test_pred)

            report[list(models.keys())[i]] = test_model_r2_score

        return report

    except Exception as e:
        raise e

In [22]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = (CONFIG_FILE_PATH),
                 params_filepath = (PARAMS_FILE_PATH),
                 schema_filepath = (SCHEME_FILE_PATH),
                 model_p_filepath = (MODEL_P_FILE_PATH)):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        self.model_p = read_yaml(model_p_filepath)

        create_directories([self.config.artifacts_root])
        print(self.config)

    def get_model_trainer_config(self) -> ModelTrainerConfig:

        config = self.config.model_trainer
        # params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        model = self.params.models
        model_p = self.model_p.params
        # print(config)
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            # alpha=params.alpha,
            # l1_ration=params.l1_ratio,
            target_column=schema.name,
            model_to_loop = model,
            model_params = model_p,
            train_array_path=config.train_array_path,
            test_array_path=config.test_array_path
        )

        return model_trainer_config



In [23]:
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib
# from mlProject.utils.common import evaluate_models
from mlProject import *
from pathlib import Path

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config  = config

    def train(self):
        # train_data = pd.read_csv(self.config.train_data_path)
        # test_data = pd.read_csv(self.config.test_data_path)

        # train_x = train_data.drop([self.config.target_column], axis=1)
        # train_y = train_data[self.config.target_column]

        # test_x = test_data.drop([self.config.target_column], axis=1)
        # test_y = test_data[self.config.target_column]
        # print(self.config)
        logger.info('Training of array started')
        train_data = load_object(Path(self.config.train_array_path))
        test_data  = load_object(Path(self.config.test_array_path))

        train_x, train_y, test_x , test_y = (
            train_data[:,:-1],
            train_data[:,-1],
            test_data[:,:-1],
            test_data[:,-1]
        )
        models = self.config.model_to_loop
        model_p = self.config.model_params
        logger.info(f"Models parameters are ready to print")
        print(model_p)
        print(model_p.keys())
        print(model_p.values())

        model_report:dict = evaluate_models(x_train=train_x, y_train=train_y, x_test=test_x, y_test=test_y, models=models, params=model_p)

        
        best_model_score = max(sorted(model_report.values()))

        best_model_name = list(model_report.keys())[
            list(model_report.values()).index(best_model_score)
            ]

        best_model = models[best_model_name]

        logging.info(f"Best model is {best_model_name} and its r2 score is {best_model_score}")

        joblib.dump(best_model, os.path.join(self.config.root_dir, self.config.model_name))

In [24]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-01-26 11:34:10,097: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-26 11:34:10,104: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-26 11:34:10,111: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-26 11:34:10,159: INFO: common: yaml file: model_params.yaml loaded successfully]
[2024-01-26 11:34:10,163: INFO: common: Created directory at artifacts]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/ashay-thamankar/raw-data-to-practice/raw/main/winequality-red.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/winequality-red.csv', 'STATUS_FILE': 'artifacts/data_validation/staus.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestio

ModelTrainerConfig(root_dir='artifacts/model_trainer', train_data_path='artifacts/data_transformation/train.csv', test_data_path='artifacts/data_transformation/test.csv', model_name='model.joblib', target_column='quality', model_to_loop=ConfigBox({'Linear Regression': 'LinearRegression()', 'K-Neighbors Regressor': 'KNeighborsRegressor()', 'Decision Tree': 'DecisionTreeRegressor()', 'Random Forest Regressor': 'RandomForestRegressor()', 'XGBRegressor': 'XGBRegressor()', 'CatBoosting Regressor': 'CatBoostRegressor(verbose=False)', 'AdaBoost Regressor': 'AdaBoostRegressor()', 'Gradient Boosting': 'GradientBoostingRegressor()', 'ElasticNet': 'ElasticNet()'}), model_params=ConfigBox({'Linear Regression': {'fit_intercept': [True, False], 'copy_X': [True, False], 'n_jobs': [-1, 1, 'None']}, 'K-Neighbors Regressor': {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'leaf_size': [10, 20, 30], 'p': [1, 2], 'metric': ['minkows

c:\Users\User\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
15 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\User\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "c:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\User\anaconda3\Lib\site-packages\sklea

{'Linear Regression': 0.4019459730722381, 'K-Neighbors Regressor': 0.5425946067391587, 'Decision Tree': -0.023326708408673014, 'Random Forest Regressor': 0.5237928006946831, 'XGBRegressor': 0.47887715253929086, 'CatBoosting Regressor': 0.4957674546314439, 'AdaBoost Regressor': 0.4173218939857646, 'Gradient Boosting': 0.43862518523491656, 'ElasticNet': 0.36196096133633715}
[2024-01-26 11:37:08,413: INFO: 3462842478: Best model is K-Neighbors Regressor and its r2 score is 0.5425946067391587]
